In [1]:
import pandas as pd
import numpy as np

pm=open('ppm_98_TMRstart.fas','r')
npm=open('npm_98_TMRstart.fas','r')

fas = [[]]

while True:
    reading = pm.readline()
    reading = reading.rstrip()

    if not reading.startswith('>'):
        if reading == '':
            break
        fas.append(list(reading))

fas.remove([])
columns=list(range(-25,26))
df_pm=pd.DataFrame(fas,columns=columns)

fas = [[]]

while True:
    reading = npm.readline()
    reading = reading.rstrip()

    if not reading.startswith('>'):
        if reading == '':
            break
        fas.append(list(reading))

fas.remove([])
columns=list(range(-25,26))
df_npm=pd.DataFrame(fas,columns=columns)

In [8]:
position = -2

df_pm_freq=df_pm[position].value_counts(sort=True)
df_pm_nor=df_pm[position].value_counts(sort=True,normalize=True)

df_npm_freq=df_npm[position].value_counts(sort=True)
df_npm_nor=df_npm[position].value_counts(sort=True,normalize=True)

printout=pd.concat([df_pm_freq,df_pm_nor,df_npm_freq,df_npm_nor],axis='columns')

printout

,-2,-2,-2,-2
K,13.0,0.166667,87,0.154529
G,12.0,0.153846,35,0.062167
R,12.0,0.153846,141,0.250444
T,9.0,0.115385,41,0.072824
S,7.0,0.089744,38,0.067496
N,7.0,0.089744,22,0.039076
P,5.0,0.064103,28,0.049734
W,4.0,0.051282,31,0.055062
H,3.0,0.038462,19,0.033748
Q,3.0,0.038462,28,0.049734
